In [72]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential

from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam

from keras.applications import xception

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

In [2]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'

gen = image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=0.1,
                               shear_range=0.15, 
                               zoom_range=0.1, 
                               channel_shift_range=10., 
                               horizontal_flip=True)
batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

### train

#### pretrained

In [48]:
batches = image.ImageDataGenerator().flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size)
batches_val = image.ImageDataGenerator().flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')

# model_x = add_preprocess(base_model, xception.preprocess_input)
# bf_x=model_x.predict_generator(batches, steps=nb_batches, verbose=1)
# np.save(data_dir+'/results/bf_x', bf_x)
bf_x = np.load(data_dir+'/results/bf_x.npy')
# bf_val_x=model_x.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_x', bf_val_x)
bf_val_x = np.load(data_dir+'/results/bf_val_x.npy')

lm = Sequential([Dense(120, activation='softmax', input_shape=(2048,))])
lm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
lm.fit(bf_x, y, epochs=15, batch_size=nb_batches, validation_data=(bf_val_x, y_val))

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 0s - loss: 3.0953 - acc: 0.6562 - val_loss: 1.9305 - val_acc: 0.8545
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 1.3740 - acc: 0.8772 - val_loss: 0.9951 - val_acc: 0.8865
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.7553 - acc: 0.9017 - val_loss: 0.6414 - val_acc: 0.8980
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.5086 - acc: 0.9096 - val_loss: 0.4893 - val_acc: 0.9020
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.3910 - acc: 0.9151 - val_loss: 0.4091 - val_acc: 0.9045
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.3231 - acc: 0.9254 - val_loss: 0.3682 - val_acc: 0.9040
Epoch 7/15
8222/8222 [==============================] - 0s - loss: 0.2798 - acc: 0.9303 - val_loss: 0.3425 - val_a

In [49]:
lm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 120)               245880    
Total params: 245,880
Trainable params: 245,880
Non-trainable params: 0
_________________________________________________________________


In [50]:
lm.layers[0]

#### augmentation

In [62]:
batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size)
batches_val = image.ImageDataGenerator().flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.


In [77]:
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
inputs = Input(shape=(299, 299, 3))
x = Lambda(xception.preprocess_input)(inputs)
x = base_model(x)
outputs = Dense(120, activation='softmax', name='predictions')(x)
model_ft = Model(inputs, outputs)
for layer in base_model.layers:
    layer.trainable = False

In [78]:
model_ft.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 299, 299, 3)       0         
_________________________________________________________________
lambda_10 (Lambda)           (None, 299, 299, 3)       0         
_________________________________________________________________
xception (Model)             (None, 2048)              20861480  
_________________________________________________________________
predictions (Dense)          (None, 120)               245880    
Total params: 21,107,360
Trainable params: 245,880
Non-trainable params: 20,861,480
_________________________________________________________________


In [79]:
model_ft.layers[3].set_weights(lm.layers[0].get_weights())

In [80]:
model_ft.compile(optimizer=Adam(lr=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

In [81]:
# verify that the starting point is already good model
model_ft.evaluate_generator(batches_val, steps=nb_batches_val)

[0.29967120851576329, 0.91049999999999998]

In [82]:
model_ft.fit_generator(batches, 
                    steps_per_epoch=nb_batches, 
                    epochs=10,
                    validation_data=batches_val,
                    validation_steps=nb_batches_val
                   )

Epoch 1/10
514/514 [==============================] - 147s - loss: 0.3868 - acc: 0.8761 - val_loss: 0.4543 - val_acc: 0.8615
Epoch 2/10
514/514 [==============================] - 144s - loss: 0.3879 - acc: 0.8739 - val_loss: 0.4547 - val_acc: 0.8620
Epoch 3/10
514/514 [==============================] - 145s - loss: 0.3898 - acc: 0.8744 - val_loss: 0.4540 - val_acc: 0.8630
Epoch 4/10
405/514 [======================>.......] - ETA: 24s - loss: 0.3852 - acc: 0.8742

KeyboardInterrupt: 

### predict

In [9]:
batches_test = image.ImageDataGenerator().flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

Found 10357 images belonging to 1 classes.


In [10]:
nb_batches_test = math.ceil(batches_test.n/batch_size)

In [11]:
pred = model_ft.predict_generator(batches_test, steps=nb_batches_test, verbose=1)

648/648 [==============================] - 159s   


In [18]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [20]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [24]:
description = 'xception_data_augmentation'
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [12]:
description = 'xception_data_augmentation'


In [8]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [13]:
!kg submit $submission_file_name -m $description

0.52258
